# EE Review
This notebook is intended to validate information in the ExtractTransformLoad table as modeled in Django.

### Import some libraries, configure Django

In [1]:
import os

import sqlite3

import shapely
from shapely.geometry import Polygon
import pandas as pd
import geopandas as gpd

import django

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from asgiref.sync import sync_to_async
from django.core.management import call_command
from django.contrib.gis.geos import GEOSGeometry

from whale.models import EarthExplorer as EE

In [2]:
imgs = await sync_to_async(list)(EE.objects.all())

for img in imgs[0:5]:
    geom = GEOSGeometry(img.bounds).wkt
    print(f"EE - X: {img.pixel_size_x} | Y: {img.pixel_size_y} | Geom: {geom}")

EE - X: 1.0 | Y: 1.0 | Geom: POLYGON ((-70.360465 41.807725, -70.360465 41.927938, -70.094004 41.927938, -70.094004 41.807725, -70.360465 41.807725))
EE - X: 0.0 | Y: 0.0 | Geom: POLYGON ((-70.360134 41.807752, -70.360134 41.927927, -70.094304 41.927927, -70.094304 41.807752, -70.360134 41.807752))
EE - X: 1.0 | Y: 1.0 | Geom: POLYGON ((-70.360829 41.870817, -70.360829 41.993979, -70.093706 41.993979, -70.093706 41.870817, -70.360829 41.870817))
EE - X: 0.0 | Y: 0.0 | Geom: POLYGON ((-70.360496 41.870846, -70.360496 41.993965, -70.094007 41.993965, -70.094007 41.870846, -70.360496 41.870846))
EE - X: 1.0 | Y: 1.0 | Geom: POLYGON ((-70.361174 41.934253, -70.361174 42.060377, -70.093235 42.060377, -70.093235 41.934253, -70.361174 41.934253))


In [3]:
db = "../../db.sqlite3"

whale_earthexplorer_columns = ['entity_id', 'catalog_id', 'acquisition_date', 'vendor',
                               'vendor_id', 'cloud_cover', 'satellite', 'sensor',
                               'number_of_bands', 'map_projection', 'datum',
                               'processing_level', 'file_format', 'license_id',
                               'sun_azimuth', 'sun_elevation', 'pixel_size_x',
                               'pixel_size_y', 'license_uplift_update', 'event',
                               'date_entered', 'center_latitude_dec',
                               'center_longitude_dec', 'thumbnail', 'publish_date',
                               'aoi_id_id', 'event_date']

conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

columns_list = list(whale_earthexplorer_columns)
columns_str = ', '.join(columns_list)
sql_string = "SELECT {}, AsText(bounds) FROM whale_earthexplorer WHERE vendor IS NOT NULL AND aoi_id_id = {}".format(columns_str, 1)

df = pd.read_sql_query(sql_string, conn)
df = df.rename(columns={'AsText(bounds)': 'geometry'}, errors='raise')
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry='geometry')

conn.commit()
conn.close()

gdf.head()

,entity_id,catalog_id,acquisition_date,vendor,vendor_id,cloud_cover,satellite,sensor,number_of_bands,map_projection,...,license_uplift_update,event,date_entered,center_latitude_dec,center_longitude_dec,thumbnail,publish_date,aoi_id_id,event_date,geometry
0,WV320240704205751P00,1040010096C03300,2024-07-04,MAXAR TECHNOLOGIES,24JUL04205751-P1BS-508530682010_02_P007,6,WORLDVIEW-3,PAN,1,GCP,...,None,UCDAM,2024-07-05,60.700664,-151.333349,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-07-05 15:57:26.517893,1,None,"POLYGON ((-151.53281 60.63548, -151.53281 60.7..."
1,WV320240704205751M00,1040010096C03300,2024-07-04,MAXAR TECHNOLOGIES,24JUL04205751-M1BS-508530682010_02_P007,6,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-07-05,60.700648,-151.333372,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-07-05 15:57:33.271479,1,None,"POLYGON ((-151.5333 60.63544, -151.5333 60.765..."
2,WV320240704205750P00,1040010096C03300,2024-07-04,MAXAR TECHNOLOGIES,24JUL04205750-P1BS-508530682010_02_P006,0,WORLDVIEW-3,PAN,1,GCP,...,None,UCDAM,2024-07-05,60.636445,-151.333375,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-07-05 15:57:23.396981,1,None,"POLYGON ((-151.53374 60.56951, -151.53374 60.7..."
3,WV320240704205750M00,1040010096C03300,2024-07-04,MAXAR TECHNOLOGIES,24JUL04205750-M1BS-508530682010_02_P006,0,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-07-05,60.636431,-151.333397,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-07-05 15:57:35.245827,1,None,"POLYGON ((-151.53424 60.56946, -151.53424 60.7..."
4,WV320240704205746M00,1040010096C03300,2024-07-04,MAXAR TECHNOLOGIES,24JUL04205746-M1BS-508530682010_02_P001,0,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-07-05,60.304872,-151.333417,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-07-05 15:57:40.549114,1,None,"POLYGON ((-151.53999 60.22808, -151.53999 60.3..."


In [4]:
set(gdf['pixel_size_y'])

{0.3, 1.4}

In [5]:
set(gdf['pixel_size_x'])

{0.3, 1.4}

In [6]:
set(gdf['aoi_id_id'])

{1}

In [7]:
type(gdf['aoi_id_id'][0])

numpy.int64